In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import urllib.parse
import csv
import pickle
import pandas as pd
import numpy as np

In [2]:
api_key = ""

In [3]:
def get_goedata(address, api_key):
    
    address = urllib.parse.quote(address)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={key}".format(city=address, key=api_key)
    city = ""
    canton = ""
    country = ""
    lat = 0
    lng = 0
    
    data = requests.get(url)
    geodata = json.loads(data.text)
    if geodata['status'] == "OK":
        for level in geodata["results"][0]["address_components"]:
            if "locality" in level["types"]:
                city = level['short_name']
            if 'administrative_area_level_1' in level["types"]:
                canton = level['short_name']
            if 'country' in level["types"]:
                country = level['short_name']
        lat = geodata["results"][0]['geometry']['location']['lat']
        lng = geodata["results"][0]['geometry']['location']['lng']

    return (city, canton, country, lat, lng)
    
    

In [5]:
df = pd.read_csv("../data/marathon-data.csv" ,sep='\t')

In [6]:
df.head(50)

,Unnamed: 0,doss,retard,nom,catégorie,rang,an,lieu,temps,race-year
0,0,(3001),"55.32,7",Abbitt Pamela,SD4,66.,50,GB-Luton,"2:30.44,4",2002
1,1,(3002),"40.25,0",Abbitt Philip,SH4,182.,52,GB-Luton,"2:02.23,2",2002
2,2,(9411),"52.40,0",Abderhalden Seth,RH,248.,44,Renens VD,"1:29.58,1",2002
3,3,(101),"47.01,7",Abdulaziz Sirwan,MH3,83.,60,Vernier,"3:13.32,2",2002
4,4,(102),"2:56.38,8",Abe Shinya,MH4,222.,49,JPN-Ibaraki 310-0066,"5:39.21,8",2002
5,5,(33),"20.33,8",Abosa Emebet,MD1,3.,77,ETH-Ethiopie,"2:49.37,0",2002
6,6,(9001),"14.54,0",Abraham Michel,RH,99.,46,F-Pontarlier,"52.12,1",2002
7,7,(9003),"9.15,1",Abt Hans,RH,39.,47,Studen BE,"46.33,2",2002
8,8,(3004),"37.04,5",Accetta-Parisi Vicenzo,SH2,340.,69,Belmont-Lausanne,"1:45.12,4",2002
9,9,(3005),"40.55,4",Aceti Vendrani Daniela,SD2,137.,68,St-Saphorin-Morges,"2:01.49,9",2002


In [7]:
df = df.replace(np.nan, '', regex=True)

In [9]:
def translate_geodata(city_list, reset_idx=False):
    city_dict = {}
    idx = None
    try:
        print('Trying to load data from pickle')
        if reset_idx:
            idx = 0
            pickle.dump(0, open('../data/last_index.p', 'wb'))
        else:
            idx = pickle.load(open('../data/last_index.p', 'rb'))
    except (OSError, IOError) as e:
        print("load data error")

    print('Starting translating from {}'.format(idx))
    try:
        city_list = city_list[idx: ]
        for city in city_list:
            if len(city):
                city_dict[city] = get_goedata(city, api_key)
            idx += 1
            if idx % 1000 == 0:
                print(idx)
        else:
            print("Finish at {}".format(idx))
            update_dict(city_dict,idx)
            
    except Exception as e:
        print(e)
        print(idx)
        update_dict(city_dict,idx)

def update_dict(city_dict,idx):
    old_dict = pickle.load(open('../data/city_geo.p', 'rb'))
    city_dict.update(old_dict)
    pickle.dump(city_dict, open('../data/city_geo.p', 'wb'))
    pickle.dump(idx, open('../data/last_index.p', 'wb'))

In [10]:
city_list = list(df['lieu'].unique())

In [27]:
translate_geodata(city_list[:6001], False)

Trying to load data from pickle
Starting translating from 5999
6000
Finish at 6001


In [11]:
city_dicts = pickle.load(open('../data/city_geo.p', 'rb'))
city_dicts[''] = ('', '', '', 0, 0)
len(city_dicts.keys())

FileNotFoundError: [Errno 2] No such file or directory: '../data/city_geo.p'

In [68]:
df['format_city'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][0])
df['Canton'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][1])
df['country'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][2])
df['lat'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][3])
df['lng'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][4])

In [1]:
df.head(10)

NameError: name 'df' is not defined

In [71]:
df.to_csv("../data/marathon-data-with-geo.csv" ,sep=',')